## An example how to calculate sdg 11.2 indicator for Friedrichshain (Berlin)

In [1]:
import os
import sys

import pandas as pd

project_path = os.path.abspath('../')
sys.path.append(project_path)
import geopandas as gpd
from matplotlib import pyplot as plt
from ptac import accessibility as accessibility

Load stops and population data and generate boundary from population data.
The boundary is necessary to download the network afterwards.

In [ ]:
stops = gpd.read_file(project_path + "/data/pt_example.gpkg")
stops_high_capacity = gpd.read_file(project_path + "/data/pt_example_high_capacity.gpkg")
stops_low_capacity = gpd.read_file(project_path + "/data/pt_example_low_capacity.gpkg")
population_points = gpd.read_file(project_path + "/data/population_example.gpkg")
boundary_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[population_points.unary_union.convex_hull])

In [ ]:
population_points.crs

Plot population and pt stops

In [ ]:
ax = population_points.plot(column= "pop", cmap='OrRd', legend=True)
stops.plot(color="blue", ax=ax, markersize=6)

Calculate sum of population in the focus area

In [ ]:
population_sum = population_points["pop"].sum()

print(f"Overall sum of population is: {population_sum}")

Now calculate the accessibilities from population point data to stations

In [ ]:
accessibility_output = accessibility.distance_to_closest(
    start_geometries=population_points,
    destination_geometries=stops,
    boundary_geometries=boundary_gdf,
    maximum_distance=500,
    epsg=25833,
    network_exists=False,
    verbose=5)

Select starting points which are within 500 meters from the next station

In [ ]:
sdg = accessibility.calculate_sdg(population=population_points,
                                  population_column='pop',
                                  input=accessibility_output)
print(f"{sdg} percent of the population have access to public transit stops")

Calculate sum of population with access to pt within 500 meters

In [ ]:
sum_pop_with_access = accessibility_output["pop"].sum()
print(sum_pop_with_access)

Plot result as a map

In [ ]:
accessibility_output.plot(column='distance_pt', cmap='OrRd')
plt.show()

In [ ]:
accessibility_output_high = accessibility.distance_to_closest(
    start_geometries=population_points,
    destination_geometries=stops_high_capacity,
    transport_system="high-capacity",
    boundary_geometries=boundary_gdf,
    epsg=25833,
    network_exists=True,
    verbose=5)

In [ ]:
accessibility_output_low = accessibility.distance_to_closest(
    start_geometries=population_points,
    destination_geometries=stops_low_capacity,
    transport_system="low-capacity",
    boundary_geometries=boundary_gdf,
    epsg=25833,
    network_exists=True,
    verbose=5)

Calculate the actual indicator for low- and high-capacity transport systems and print the result

In [ ]:
sdg_low_high = accessibility.calculate_sdg(population=population_points,
                                           population_column='pop',
                                           input=[accessibility_output_high,
                                                  accessibility_output_low])
print(f"{sdg_low_high} percent of the population have access to low- and high-capacity public transit systems")


In [ ]:
accessibility_low_high = accessibility.concat_dfs(input=[accessibility_output_high, accessibility_output_low])

In [ ]:
accessibility_low_high.plot(column='distance_pt', cmap='OrRd')
plt.show()

In [ ]:
accessibility.create_population_data(input_path="D:/trak/cities",
                                     output_path="D:/trak/cities_population_point", extension=".csv")
